# Convert TSV Data To Parquet with Athena

In this notebook, we will show you how you can easily convert that data now into Apache Parquet file format.

<img src="img/athena_convert_parquet.png" width="60%" align="left">

In [1]:
import boto3
import sagemaker

sess = sagemaker.Session()
bucket = sess.default_bucket()
role = sagemaker.get_execution_role()
region = boto3.Session().region_name

In [2]:
ingest_create_athena_table_parquet_passed = False

In [3]:
%store -r ingest_create_athena_table_tsv_passed

In [4]:
try:
    ingest_create_athena_table_tsv_passed
except NameError:
    print("++++++++++++++++++++++++++++++++++++++++++++++")
    print("[ERROR] YOU HAVE TO RUN ALL PREVIOUS NOTEBOOKS.  You did not register the TSV Data.")
    print("++++++++++++++++++++++++++++++++++++++++++++++")

In [5]:
print(ingest_create_athena_table_tsv_passed)

True


In [6]:
if not ingest_create_athena_table_tsv_passed:
    print("++++++++++++++++++++++++++++++++++++++++++++++")
    print("[ERROR] YOU HAVE TO RUN ALL PREVIOUS NOTEBOOKS.  You did not register the TSV Data.")
    print("++++++++++++++++++++++++++++++++++++++++++++++")
else:
    print("[OK]")

[OK]


# Import PyAthena

In [7]:
from pyathena import connect

# Create Parquet Files from TSV Table

As you can see from the query below, we’re also adding a new `year` column to our dataset by converting the `review_date` string to a date format, and then cast the year out of the date. Let’s store the year value as an integer. And let's partition the Parquet data by `Product Category`.

In [8]:
# Set S3 path to Parquet data
s3_path_parquet = "s3://{}/amazon-reviews-pds/parquet".format(bucket)

# Set Athena parameters
database_name = "dsoaws"
table_name_tsv = "amazon_reviews_tsv"
table_name_parquet = "amazon_reviews_parquet"

In [9]:
# Set S3 staging directory -- this is a temporary directory used for Athena queries
s3_staging_dir = "s3://{0}/athena/staging".format(bucket)

In [10]:
conn = connect(region_name=region, s3_staging_dir=s3_staging_dir)

# Execute Statement
_This can take a few minutes.  Please be patient._

In [11]:
# SQL statement to execute
statement = """CREATE TABLE IF NOT EXISTS {}.{}
WITH (format = 'PARQUET', external_location = '{}', partitioned_by = ARRAY['product_category']) AS
SELECT marketplace,
         customer_id,
         review_id,
         product_id,
         product_parent,
         product_title,
         star_rating,
         helpful_votes,
         total_votes,
         vine,
         verified_purchase,
         review_headline,
         review_body,
         CAST(YEAR(DATE(review_date)) AS INTEGER) AS year,
         DATE(review_date) AS review_date,
         product_category
FROM {}.{}""".format(
    database_name, table_name_parquet, s3_path_parquet, database_name, table_name_tsv
)

print(statement)

CREATE TABLE IF NOT EXISTS dsoaws.amazon_reviews_parquet
WITH (format = 'PARQUET', external_location = 's3://sagemaker-us-east-1-211125778552/amazon-reviews-pds/parquet', partitioned_by = ARRAY['product_category']) AS
SELECT marketplace,
         customer_id,
         review_id,
         product_id,
         product_parent,
         product_title,
         star_rating,
         helpful_votes,
         total_votes,
         vine,
         verified_purchase,
         review_headline,
         review_body,
         CAST(YEAR(DATE(review_date)) AS INTEGER) AS year,
         DATE(review_date) AS review_date,
         product_category
FROM dsoaws.amazon_reviews_tsv


In [12]:
import pandas as pd

pd.read_sql(statement, conn)

,rows


# Load partitions by running `MSCK REPAIR TABLE`

As a last step, we need to load the Parquet partitions. To do so, just issue the following SQL command: 

In [13]:
statement = "MSCK REPAIR TABLE {}.{}".format(database_name, table_name_parquet)

print(statement)

MSCK REPAIR TABLE dsoaws.amazon_reviews_parquet


In [14]:
import pandas as pd

df = pd.read_sql(statement, conn)
df.head(5)

""


# Show the Partitions

In [15]:
statement = "SHOW PARTITIONS {}.{}".format(database_name, table_name_parquet)

print(statement)

SHOW PARTITIONS dsoaws.amazon_reviews_parquet


In [16]:
df_partitions = pd.read_sql(statement, conn)
df_partitions.head(5)

,partition
0,product_category=Digital_Video_Games
1,product_category=Gift Card
2,product_category=Digital_Software


# Show the Tables

In [17]:
statement = "SHOW TABLES in {}".format(database_name)

In [18]:
df_tables = pd.read_sql(statement, conn)
df_tables.head(5)

,tab_name
0,amazon_reviews_parquet
1,amazon_reviews_tsv


In [19]:
if table_name_parquet in df_tables.values:
    ingest_create_athena_table_parquet_passed = True

In [20]:
%store ingest_create_athena_table_parquet_passed

Stored 'ingest_create_athena_table_parquet_passed' (bool)


# Run Sample Query

In [21]:
product_category = "Digital_Software"

statement = """SELECT * FROM {}.{}
    WHERE product_category = '{}' LIMIT 100""".format(
    database_name, table_name_parquet, product_category
)

print(statement)

SELECT * FROM dsoaws.amazon_reviews_parquet
    WHERE product_category = 'Digital_Software' LIMIT 100


In [22]:
df = pd.read_sql(statement, conn)
df.head(5)

,marketplace,customer_id,review_id,product_id,product_parent,product_title,star_rating,helpful_votes,total_votes,vine,verified_purchase,review_headline,review_body,year,review_date,product_category
0,US,10495702,R3TKUASN3SL16O,B00MEUPSXS,625882117,QuickBooks Premier 2015,4,1,1,N,Y,QuickBooks is a good program but this version ...,QuickBooks is a good program but this version ...,2015,2015-08-18,Digital_Software
1,US,9682914,RE85JCBUDBPKR,B00SG9ABQU,303901870,Pluto TV: 100+ Free Channels [Download],1,0,0,N,Y,One Star,did not work. at all,2015,2015-08-18,Digital_Software
2,US,49441252,R2P2G6RSG65GCH,B00IT6HE5G,228166066,"IObit Malware Fighter 2 Pro, 1 Year / 1 PC [Do...",1,0,0,N,Y,One Star,do not installe this it causes you computer to...,2015,2015-08-18,Digital_Software
3,US,47046980,R2UZJRYK8AGEZC,B00P31G9PQ,831433899,Amazon Drive Desktop [PC],3,1,1,N,Y,eh.,It saves my photos...over and over and over ag...,2015,2015-08-18,Digital_Software
4,US,6626191,R23T84T2VBZHQ7,B00C1QCBY4,640306152,Sketchbook Pro 6,5,0,0,N,Y,Five Stars,Fast and speedy delivery,2015,2015-08-18,Digital_Software


In [23]:
if not df.empty:
    print("[OK]")
else:
    print("++++++++++++++++++++++++++++++++++++++++++++++++++++++")
    print("[ERROR] YOUR DATA HAS NOT BEEN CONVERTED TO PARQUET. LOOK IN PREVIOUS CELLS TO FIND THE ISSUE.")
    print("++++++++++++++++++++++++++++++++++++++++++++++++++++++")

[OK]


# Review the New Athena Table in the Glue Catalog

In [24]:
from IPython.core.display import display, HTML

display(
    HTML(
        '<b>Review <a target="top" href="https://console.aws.amazon.com/glue/home?region={}#">AWS Glue Catalog</a></b>'.format(
            region
        )
    )
)

In just a few steps we have set up Amazon Athena to connect to our Amazon Customer Reviews TSV files, and transformed them into Apache Parquet file format. 

You might have noticed that our second sample query finished in a fraction of the time compared to the one before we ran on the TSV table. We sped up our query results by leveraging our data being stored as Parquet and partitioned by `product_category`. 


# Store Variables for the Next Notebooks

In [25]:
%store

Stored variables and their in-db values:
auto_ml_job_name                                      -> 'automl-dm-18-14-40-14'
autopilot_endpoint_arn                                -> 'arn:aws:sagemaker:us-east-1:211125778552:endpoint
autopilot_endpoint_name                               -> 'automl-dm-ep-18-14-35-04'
autopilot_model_arn                                   -> 'arn:aws:sagemaker:us-east-1:211125778552:model/au
autopilot_model_name                                  -> 'automl-dm-model-18-14-35-04'
autopilot_train_s3_uri                                -> 's3://sagemaker-us-east-1-211125778552/data/amazon
comprehend_endpoint_arn                               -> 'arn:aws:comprehend:us-east-1:211125778552:documen
comprehend_train_s3_uri                               -> 's3://sagemaker-us-east-1-211125778552/data/amazon
comprehend_training_job_arn                           -> 'arn:aws:comprehend:us-east-1:211125778552:documen
ingest_create_athena_db_passed                        -> Tr

# Release Resources

In [26]:
%%html

<p><b>Shutting down your kernel for this notebook to release resources.</b></p>
<button class="sm-command-button" data-commandlinker-command="kernelmenu:shutdown" style="display:none;">Shutdown Kernel</button>
        
<script>
try {
    els = document.getElementsByClassName("sm-command-button");
    els[0].click();
}
catch(err) {
    // NoOp
}    
</script>

In [27]:
%%javascript

try {
    Jupyter.notebook.save_checkpoint();
    Jupyter.notebook.session.delete();
}
catch(err) {
    // NoOp
}

<IPython.core.display.Javascript object>